# Paging

We want to be able to dynamically allocate memory, but also ensure that physical memory will not become fragmented.

The basic idea is that we break physical memory into equal size chunks which we can give out as needed.

We have two memory spaces to keep in mind, the Virtual Address Space and teh Physical Memory.

We break the Virtual Address Space (VAS) into **virtual pages**.

For example, a 64 byte address space:

<br>
<img src="images/07-paging-VAS.png" width="500">
<br>

We break physical memory into **physical frames**.

We map virtual pages to physical frames.

`VP_0` has to be in some physical frame in memory. Virtual pages and physical frames are of the same size since we are creating a one to one mapping from virtual pages to physical frames.

<br>
<img src="images/08-paging-phys-mem.png" width="500">
<br>

The OS stores the mapping from virtual pages to the physical frame they are stored on in a **page table**.

The page table is just an array. 

Index 0 tells us which physical frame `VP_0` is stored at. 

Index 1 tells us which physical frame `VP_1` is stored at.

The OS keeps a page table for every process.

This is stored in OS memory. There is a specific register that holds the memory address of the beginning of this array.

This register is called the **Page Table Base Register (PTBR)**.

## Anatomy of a Virtual Address

Each VA must contain:

- which virtual page this address is referencing, the Virtual Page Number (VPN)
- how far into that virtual page the memory address is, OFFSET

<br>
<img src="images/09-paging-VA.png" width="300">
<br>

With a 64 byte address space, we need 6 bits to distincly refer to every byte.

If we have 4 total pages, then we need two bits to serve as the VPN, leaving 4 bits as the OFFSET. With those 4 bits we can see that each page is 2^4 = 16 bytes large.

Example:
```
VA: 111010
VA: 11 | 1010
   VPN   OFFSET
VPN: 3
OFFSET 10
```

## Address Translation

When a process requests some Virtual Address, the OS 

1) breaks it into its two parts, the VPN and OFFSET
2) Looks up in the page table which Physical Frame this Virtual Page is stored in
3) Calculates the physical address by concatenating the OFFSET to the Virtual Frame Number

<br>
<img src="images/10-VA-to-PA.png" width="400">
<br>

<br>
<img src="images/11-phys-mem.png" width="500">
<br>

## The Page Table

The page table is an array of Page Table Entries (PTE).

Every PTE contains the Physical Number that this page maps to and extra bits of information such as is the frame valid (has it been allocated to this process yet), or is it protected (kernel mode needed to access it), or is it in RAM or swapped to disk.

<br>
<img src="images/12-PTE.png" width="500">
<br>

The 0th entry of the page table hold the PFN for VP0 for a process, and so on.

To Perform an Address Translation:

<br>
<img src="images/13-paging-mem-access.png" width="500">
<br>

## A couple of Problems

Now every memory access requires two memory access. Too slow.

The size of the page table is also problematic.

## A Real Address Space

The sieze of a virtual address space is determined in part by whether the machine is 32-bit, 64-bit, or otherwise.

32-bit and 64-bits refer to the size of the CPU registers.

Nice virtual addresses must be stored in registers, the size of the register limits how large a virtual address can be.

Standard page sizes are 4KB or 4096 bytes.

4KB seems to hit the sweet spot between being able to allocate memory is large enough chunks that it will satisfy a program, but not so large that alot of space is wasted by having many of these pages with only a few bytes in them.

With pages of size 4096 bytes, how large of an OFFSET do we need to fully address into them?

$2^{12} = 4096$

$2^{10} = 1024 = $ 1 KB

$2^{20} = $ 1 MB

$2^{30} = $ 1 GB

$2^{12} = 2^2 * 2^{10} = 4 * 1024 = 4KB$

In a 32-bit address space, 12-bits will be used for the OFFSET, leaving 20 bits for the VPN.

With 20 bits, we have $2^{20}$ virtual pages.

$2^{20} = 1048576$ pages

That means that a page table will have 1048576 Page Table Entries (PTEs) since we have to have an index for every virtual page.

## Size of the Page Table

We know we have 1048576 Page Table Entries (PTEs).

Every PTE is 32 bits, 4 bytes.

$1048576 * \text{4 bytes} = \text{4194304 bytes}$

$= \text{4 MB}$

The OS must store a page table for every running process.

If we have 1000 processes, we need 4000 MB (~4GB) of space just to store the page tables.

Our page tables are too large.

In practice a process will only need a fraction of its max possible address space. It will only need to refer to pages that have been allocated to it.

Our page table creates entries for every possible page. If we could only store what we are actually using, we could save alot of space.

## 64-bit address spaces

A 32-bit address space allows us address up to 4GB of memory.

2^32 = 4GB

If a 32-bit address space is the size of a tennis court, a 64-bit address space is about the size of Europe.

Its so big we don't need that much space. Modern computers use 48 of the bits for virtual addresses.

A 48-bit address space is

$2^{48} = 2^{8} * 2^{40} = 256$ TB